# Penalaan Halus Model Open AI

Notebook ini berdasarkan panduan semasa yang diberikan dalam dokumentasi [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) dari Open AI.

Penalaan halus meningkatkan prestasi model asas untuk aplikasi anda dengan melatih semula model tersebut menggunakan data tambahan dan konteks yang berkaitan dengan kes penggunaan atau senario tertentu. Perlu diingat bahawa teknik kejuruteraan prompt seperti _few shot learning_ dan _retrieval augmented generation_ membolehkan anda menambah baik prompt lalai dengan data yang relevan untuk meningkatkan kualiti. Namun, pendekatan ini terhad oleh saiz tetingkap token maksimum model asas yang digunakan.

Dengan penalaan halus, kita sebenarnya melatih semula model itu sendiri dengan data yang diperlukan (membolehkan kita menggunakan lebih banyak contoh berbanding had tetingkap token maksimum) - dan melancarkan versi _kustom_ model yang tidak lagi memerlukan contoh diberikan semasa inferens. Ini bukan sahaja meningkatkan keberkesanan reka bentuk prompt kita (kita mempunyai lebih banyak fleksibiliti untuk menggunakan tetingkap token bagi tujuan lain) malah berpotensi mengurangkan kos (dengan mengurangkan bilangan token yang perlu dihantar ke model semasa inferens).

Penalaan halus mempunyai 4 langkah:
1. Sediakan data latihan dan muat naik.
1. Jalankan tugasan latihan untuk mendapatkan model yang telah ditala halus.
1. Nilai model yang telah ditala halus dan ulangi untuk kualiti.
1. Gunakan model yang telah ditala halus untuk inferens apabila berpuas hati.

Perlu diingat bahawa tidak semua model asas menyokong penalaan halus - [semak dokumentasi OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) untuk maklumat terkini. Anda juga boleh menala halus model yang telah ditala halus sebelum ini. Dalam tutorial ini, kita akan menggunakan `gpt-35-turbo` sebagai model asas sasaran untuk penalaan halus.

---


### Langkah 1.1: Sediakan Set Data Anda

Mari kita bina chatbot yang membantu anda memahami jadual berkala unsur dengan menjawab soalan tentang sesuatu unsur dalam bentuk limerik. Dalam tutorial _mudah_ ini, kita hanya akan cipta set data untuk melatih model dengan beberapa contoh respons yang menunjukkan format data yang diharapkan. Untuk kegunaan sebenar, anda perlu cipta set data dengan lebih banyak contoh. Anda juga boleh menggunakan set data terbuka (untuk domain aplikasi anda) jika ada, dan ubah suai formatnya untuk digunakan dalam proses penalaan lanjut.

Oleh kerana kita fokus pada `gpt-35-turbo` dan mahukan respons satu pusingan (penyelesaian chat), kita boleh cipta contoh menggunakan [format yang disyorkan ini](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) yang memenuhi keperluan penyelesaian chat OpenAI. Jika anda menjangkakan kandungan perbualan berbilang pusingan, anda perlu gunakan [format contoh berbilang pusingan](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) yang menyertakan parameter `weight` untuk menandakan mesej mana yang patut digunakan (atau tidak) dalam proses penalaan lanjut.

Kita akan gunakan format satu pusingan yang lebih mudah untuk tutorial ini. Data disimpan dalam [format jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) dengan 1 rekod setiap baris, setiap satu diwakili sebagai objek berformat JSON. Contoh di bawah menunjukkan 2 rekod sebagai sampel - lihat [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) untuk set contoh penuh (10 contoh) yang akan digunakan dalam tutorial penalaan lanjut kita. **Nota:** Setiap rekod _mesti_ ditulis dalam satu baris sahaja (bukan dibahagi ke beberapa baris seperti dalam fail JSON yang diformatkan)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Untuk kegunaan sebenar, anda perlu set contoh yang jauh lebih besar untuk hasil yang baik - pertimbangannya adalah antara kualiti respons dan masa/kos untuk penalaan lanjut. Kita gunakan set kecil supaya proses penalaan lanjut dapat disiapkan dengan cepat untuk tujuan ilustrasi. Lihat [contoh OpenAI Cookbook ini](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) untuk tutorial penalaan lanjut yang lebih kompleks.


### Langkah 1.2 Muat Naik Set Data Anda

Muat naik data menggunakan Files API [seperti yang diterangkan di sini](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Perlu diingat bahawa untuk menjalankan kod ini, anda mesti telah melakukan langkah-langkah berikut terlebih dahulu:
 - Memasang pakej Python `openai` (pastikan anda menggunakan versi >=0.28.0 untuk ciri-ciri terkini)
 - Tetapkan pembolehubah persekitaran `OPENAI_API_KEY` kepada kunci API OpenAI anda
Untuk maklumat lanjut, lihat [Panduan Persediaan](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) yang disediakan untuk kursus ini.

Sekarang, jalankan kod untuk mencipta fail yang akan dimuat naik daripada fail JSONL tempatan anda.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Langkah 2.1: Cipta kerja Penalaan Halus dengan SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Langkah 2.2: Semak Status kerja

Berikut adalah beberapa perkara yang anda boleh lakukan dengan API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Senaraikan n kerja fine-tuning yang terakhir
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Dapatkan butiran kerja fine-tuning tertentu
- `client.fine_tuning.jobs.cancel(<job_id>)` - Batalkan kerja fine-tuning
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Senaraikan sehingga n acara daripada kerja tersebut
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Langkah pertama dalam proses ini ialah _mengesahkan fail latihan_ untuk memastikan data berada dalam format yang betul.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Langkah 2.3: Jejak acara untuk memantau kemajuan


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Langkah 2.4: Lihat status dalam Papan Pemuka OpenAI


Anda juga boleh melihat status dengan melayari laman web OpenAI dan meneroka bahagian _Fine-tuning_ di platform tersebut. Di sini, anda boleh melihat status tugasan semasa, serta menjejak sejarah pelaksanaan tugasan sebelum ini. Dalam tangkapan skrin ini, anda dapat lihat bahawa pelaksanaan sebelum ini gagal, dan percubaan kedua berjaya. Untuk makluman, ini berlaku apabila percubaan pertama menggunakan fail JSON dengan rekod yang tidak diformatkan dengan betul - selepas dibaiki, percubaan kedua selesai dengan jayanya dan model tersebut tersedia untuk digunakan.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.ms.png)


Anda juga boleh melihat mesej status dan metrik dengan menatal ke bawah lagi dalam papan pemuka visual seperti yang ditunjukkan:

| Mesej | Metrik |
|:---|:---|
| ![Mesej](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.ms.png) |  ![Metrik](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.ms.png)|


### Langkah 3.1: Dapatkan ID & Uji Model yang Telah Disesuaikan dalam Kod


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Langkah 3.2: Muat & Uji Model Fine-Tuned di Playground

Anda kini boleh menguji model yang telah di-fine-tune dengan dua cara. Pertama, anda boleh lawati Playground dan gunakan menu drop-down Models untuk memilih model fine-tuned anda daripada senarai yang tersedia. Pilihan lain ialah menggunakan pilihan "Playground" yang dipaparkan dalam panel Fine-tuning (rujuk tangkapan skrin di atas) yang akan melancarkan paparan _perbandingan_ di mana versi model asas dan model fine-tuned dipaparkan sebelah-menyebelah untuk penilaian pantas.

Isikan sahaja konteks sistem yang digunakan dalam data latihan anda dan berikan soalan ujian anda. Anda akan perasan kedua-dua bahagian dikemas kini dengan konteks dan soalan yang sama. Jalankan perbandingan dan anda akan dapat melihat perbezaan output antara kedua-duanya. _Perhatikan bagaimana model fine-tuned menghasilkan respons dalam format yang anda sediakan dalam contoh anda, manakala model asas hanya mengikut arahan sistem._

Anda juga akan perasan bahawa perbandingan ini turut memaparkan bilangan token untuk setiap model, serta masa yang diambil untuk inferens. **Contoh khusus ini adalah contoh mudah yang bertujuan untuk menunjukkan proses sahaja dan bukan mencerminkan set data atau situasi dunia sebenar**. Anda mungkin perasan kedua-dua sampel menunjukkan bilangan token yang sama (konteks sistem dan arahan pengguna adalah sama) dengan model fine-tuned mengambil masa lebih lama untuk inferens (model tersuai).

Dalam situasi sebenar, anda tidak akan menggunakan contoh mudah seperti ini, tetapi akan melakukan fine-tuning menggunakan data sebenar (contohnya, katalog produk untuk khidmat pelanggan) di mana kualiti respons akan lebih ketara. Dalam konteks _itu_, untuk mendapatkan kualiti respons yang setara dengan model asas memerlukan lebih banyak kerja kejuruteraan prompt tersuai yang akan meningkatkan penggunaan token dan mungkin juga masa pemprosesan untuk inferens. _Untuk mencubanya, lihat contoh fine-tuning dalam OpenAI Cookbook sebagai permulaan._



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila maklum bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang sah. Untuk maklumat penting, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
